In [177]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth


def MeanScan(CAS_ID, cas_index_list , scan_list , bandwidth ) :
    # Y has index number of data frame
    Y = np.array(zip(cas_index_list,np.zeros(len(cas_index_list))), dtype=np.int)    
    X = np.array(zip(scan_list,np.zeros(len(scan_list))), dtype=np.float)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(X)
    # label of item
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)
    # Let be check
    result = []
    for k in range(n_clusters_):
        my_members = labels == k
        result.append( [CAS_ID , k , list(Y[my_members, 0]) , list(X[my_members, 0])] )
        # CAS-ID  / sub cluster / dataframe index / real scan value
    return result

In [178]:
import pandas as pd


CAS = pd.DataFrame.from_csv("CAS_renext.csv")

CAS['RI'] = CAS['RI'].apply(lambda x: x*100)
CAS = CAS[CAS["RI"] > 0 ]
CAS75 = CAS[ CAS['Quality']>= 75]
print "Number of CAS-ID : ",len(CAS75.CAS_ID.unique())
print CAS75.shape
# CAS75.head()

Power_len = len(CAS75.CAS_ID.unique())

Number of CAS-ID :  399
(6928, 12)


In [179]:
Top_Done_CAS_dic = {}
Top_Done_CAS = []
bandwidth = 30

x_index = []
y_index = []
for i in range(Power_len) :
    print i
    x_index.append(i)
    y = []
    PK_group = CAS75.groupby("Peak_ID").groups
    
    for pkID in PK_group :
        y.append([pkID,PK_group[pkID]])
    # "~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    test_pk_df = pd.DataFrame(y)
    test_pk_df = test_pk_df.rename(columns={0: 'Peak_ID', 1: 'Sample_index'})
    # "++++++++++++++++++++++++++++++++++++++++++++++++"
    x = []
    CAS_group = CAS75.groupby("CAS_ID").groups
    for cas in CAS_group :
        x.append([cas,CAS_group[cas]])
    
    "++++++++++++++++++++++++++++++++++++++++++++++++"
    pre_group = []
    for sub in CAS_group :
        CAS_ID = sub
        cas_index_list = CAS_group[sub] 
        RI_list = list( CAS75.ix[ cas_index_list ].RI.values )
        L = MeanScan( CAS_ID ,cas_index_list ,RI_list ,bandwidth) 
        pre_group = pre_group + L

    # Put all list to pre_group
    GroupedCAS = []

    for sgro in pre_group :
        # CAS-ID  / sub cluster / dataframe index / count() / scan number mean / Quality mean / Quality max
        casID = sgro[0]
        subcl_num = sgro[1]
        indeData = sgro[2]
        RIavg = np.mean(sgro[3])
        G_sub = CAS75.ix[indeData].Quality
        GroupedCAS.append( [casID , subcl_num , indeData , len(indeData) , RIavg , G_sub.mean() , G_sub.max()] )

    "~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    GroupedCAS_Df = pd.DataFrame(GroupedCAS)
    GroupedCAS_Df = GroupedCAS_Df.rename(columns={0: 'CAS_ID', 1: 'Sub_Cluster' ,2: "Sample_index" , 3: "Count",\
                                                 4: "RI_Value_avg" , 5: "QualMean", 6: "QualMax"})
    # GroupedCAS_Df

    Top_CAS_Fram = GroupedCAS_Df.sort_index(by=['QualMax',"Count","QualMean"], ascending=[False,False,False])
    
    for i in Top_CAS_Fram.values.tolist() :
        if i[0] in Top_Done_CAS_dic :
            continue
        else :
            Top_Done_CAS_dic[i[0]] = i[2]
            Top_Done_CAS.append(i[0])
            m = Top_Done_CAS_dic[i[0]]
            M = CAS_group[i[0]]
            break

    for i in test_pk_df.Sample_index :
        if len(set(i) & set(m)) >0 :
            M = M + i

    not_CAS = list(set(M) - set(m))
    CAS75 = CAS75.drop(not_CAS)

    print CAS75.shape[0]
    y_index.append(CAS75.shape[0])


0


/home/sscc_user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:50: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


6878
1
6818
2
6807
3
6805
4
6779
5
6775
6
6748
7
6745
8
6744
9
6721
10
6704
11
6668
12
6630
13
6526
14
6483
15
6446
16
6363
17
6303
18
6297
19
6280
20
6270
21
6267
22
6257
23
6254
24
6254
25
6241
26
6235
27
6214
28
6195
29
6188
30
6162
31
6156
32
6143
33
6129
34
6120
35
6110
36
6110
37
6085
38
6084
39
6027
40
6002
41
6000
42
6000
43
5998
44
5998
45
5998
46
5988
47
5923
48
5874
49
5856
50
5848
51
5823
52
5783
53
5782
54
5772
55
5763
56
5757
57
5751
58
5735
59
5691
60
5687
61
5679
62
5678
63
5675
64
5665
65
5660
66
5659
67
5659
68
5658
69
5656
70
5655
71
5654
72
5645
73
5614
74
5601
75
5593
76
5570
77
5569
78
5528
79
5528
80
5473
81
5472
82
5448
83
5438
84
5433
85
5432
86
5430
87
5429
88
5427
89
5425
90
5414
91
5407
92
5397
93
5392
94
5391
95
5391
96
5391
97
5390
98
5303
99
5291
100
5290
101
5287
102
5286
103
5285
104
5285
105
5285
106
5285
107
5285
108
5285
109
5284
110
5282
111
5277
112
5275
113
5259
114
5257
115
5128
116
5084
117
5077
118
5076
119
5072
120
5018
121
4928
122
4903
123
4

# after Reducing

In [180]:
CAS75

,Class,Sample,Date,Peak,scan_number,Intensity,Area%,CAS_ID,Quality,Peak_ID,RI,Sample_ID
9,CU,CU3,2015/11/03,6,2166,199396,0.26,000123-72-8,78.0,CU3-1103-6,661.946330,CU3-1103
11,CU,CU3,2015/11/03,8,2213,49279,0.07,000534-22-5,76.0,CU3-1103-8,665.706447,CU3-1103
12,CU,CU3,2015/11/03,9,2254,769124,1.02,000109-99-9,90.0,CU3-1103-9,669.344915,CU3-1103
13,CU,CU3,2015/11/03,10,2270,78929,0.10,000109-99-9,80.0,CU3-1103-10,670.884203,CU3-1103
15,CU,CU3,2015/11/03,12,2351,133877,0.18,000071-43-2,91.0,CU3-1103-12,679.866611,CU3-1103
16,CU,CU3,2015/11/03,13,2365,453905,0.60,000071-36-3,80.0,CU3-1103-13,681.636131,CU3-1103
20,CU,CU3,2015/11/03,15,2553,492453,0.65,000109-60-4,83.0,CU3-1103-15,716.686967,CU3-1103
22,CU,CU3,2015/11/03,17,2663,1524114,2.01,000137-32-6,83.0,CU3-1103-17,755.772358,CU3-1103
24,CU,CU3,2015/11/03,18,2689,82323,0.11,001115-11-3,90.0,CU3-1103-18,769.257951,CU3-1103
25,CU,CU3,2015/11/03,19,2738,42397,0.06,024587-26-6,97.0,CU3-1103-19,802.109705,CU3-1103


In [181]:
print len(Top_Done_CAS)
# print Top_Done_CAS

316


In [182]:
y = []
PK_group = CAS75.groupby("Peak_ID").groups


for pkID in PK_group :
    y.append([pkID,PK_group[pkID]])
"~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
test_pk_df = pd.DataFrame(y)
test_pk_df = test_pk_df.rename(columns={0: 'Peak_ID', 1: 'Sample_index'})

"++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
print "Size : ",test_pk_df.size
test_pk_df.shape

Size :  7644


(3822, 2)

In [184]:
x = []
CAS_group = CAS75.groupby("CAS_ID").groups

# print "CAS-Group type : ",type(CAS_group)


for cas in CAS_group :
    x.append([cas,CAS_group[cas]])

pre_group = []
bandwidth = 30 # 

for sub in CAS_group :
    CAS_ID = sub
    cas_index_list = CAS_group[sub] 
    RI_list = list( CAS75.ix[ cas_index_list ].RI.values )
    L = MeanScan( CAS_ID ,cas_index_list ,RI_list ,bandwidth) 
    pre_group = pre_group + L

# Put all list to pre_group
GroupedCAS = []

for sgro in pre_group :
    # CAS-ID  / sub cluster / dataframe index / count() / scan number mean / Quality mean / Quality max
    casID = sgro[0]
    subcl_num = sgro[1]
    indeData = sgro[2]
    RIavg = np.mean(sgro[3])
    G_sub = CAS75.ix[indeData].Quality
    GroupedCAS.append( [casID , subcl_num , indeData , len(indeData) , RIavg , G_sub.mean() , G_sub.max()] )

"~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
GroupedCAS_Df = pd.DataFrame(GroupedCAS)
GroupedCAS_Df = GroupedCAS_Df.rename(columns={0: 'CAS_ID', 1: 'Sub_Cluster' ,2: "Sample_index" , 3: "Count",\
                                             4: "RI_Value_avg" , 5: "QualMean", 6: "QualMax"})
print GroupedCAS_Df.shape
print len(GroupedCAS_Df.CAS_ID.unique())
GroupedCAS_Df.head()

(316, 7)
316


,CAS_ID,Sub_Cluster,Sample_index,Count,RI_Value_avg,QualMean,QualMax
0,000100-66-3,0,"[67, 223, 468, 783, 1472, 2238, 2638, 3180, 33...",62,925.246641,94.580645,97.0
1,000104-76-7,0,[1888],1,1043.400713,80.000000,80.0
2,000105-68-0,0,"[78, 354, 478, 1148, 1321, 1482, 2649, 2999, 3...",60,1145.748662,86.633333,90.0
3,000100-52-7,0,"[1871, 2242, 4361, 5927, 8263, 8736, 9244, 111...",24,1045.989817,89.208333,96.0
4,000112-52-7,0,[15159],1,1650.678733,80.000000,80.0


In [185]:
GroupedCAS_Df_re = GroupedCAS_Df[ GroupedCAS_Df['Count']>= 3]
print " Uniq Compound"
GroupedCAS_Df_re.shape

 Uniq Compound


(145, 7)

In [186]:
Final_index = []
for i in GroupedCAS_Df_re.values.tolist() :
    Final_index = Final_index + i[2]
    

Final_df = CAS75.ix[ Final_index ]
print Final_df.shape

(3606, 12)


In [187]:
Final_df.to_csv("Final_CAS.csv")